# データの読み込み,可視化 と Augmentationの実行

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt

In [3]:
# 自作クラスの読み込み
import sys
sys.path.append('../src/preprocess')
sys.path.append('../src/dataset')
from augmentation import ObjectDetectionAugmentation
from dataset import Dataset

- yamlファイルからconfigの読み込み

In [4]:
import yaml
with open('../config/config_train.yml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

- train, test csvの読み込み

In [34]:
root_dir  = '../input/tensorflow-great-barrier-reef'
train_path = f'{root_dir}/train.csv'
test_path = f'{root_dir}/test.csv'

def get_img_path(df):
    df['image_path'] = f'{root_dir}/train_images/video_{df.video_id}/{df.video_frame}.jpg'
    return df

def get_annotes(df):
    df['dict_annotations'] = df['annotations'].apply(lambda x: eval(x)) # annotationsが文字列として入っているため、eval()
    df['list_annotations'] = df['dict_annotations'].apply(lambda x: [list(annote.values()) for annote in x]) # リスト型式にcast
    df['n_annotations'] = df['dict_annotations'].apply(len)
    return df

def read_train(train_path):
    train = pd.read_csv(train_path)
    train = get_annotes(train)
    train = get_img_path(train)
    return train

In [35]:
train = read_train(train_path)

In [38]:
list(train.list_annotations)

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [[559, 213, 50, 32]],
 [[558, 213, 50, 32]],
 [[557, 213, 50, 32]],
 [[556, 214, 50, 32]],
 [[555, 214, 50, 32]],
 [[550, 214, 50, 32]],
 [[538, 209, 56, 47]],
 [[535, 207, 58, 48]],
 [[532, 205, 59, 49]],
 [[529, 200, 61, 50]],
 [[525, 196, 63, 51]],
 [[522, 191, 64, 52]],
 [[519, 187, 66, 52]],
 [[519, 181, 68, 53]],
 [[518, 176, 69, 54]],
 [[518, 171, 71, 55]],
 [[518, 165, 73, 56]],
 [[519, 160, 74, 58]],
 [[519, 155, 76, 60]],
 [[520, 151, 78, 62], [598, 204, 58, 32]],
 [[520, 148, 80, 63], [598, 206, 61, 34]],
 [[521, 144, 82, 65], [599, 208, 64, 36]],
 [[522, 141, 83, 67], [599, 209, 66, 38]],
 [[521, 139, 85, 69], [599, 211, 69, 40]],
 [[521, 138, 87, 71], [599, 213, 72, 42]],
 [[521, 138, 89, 73], [602, 219, 72, 42]],
 [[521, 139, 91, 74], [604, 226, 72, 42]],
 [[523, 134, 92, 77], [607, 232, 72, 42]],
 [[524, 139, 94, 80], [609, 238, 72, 42]],
 [[525, 145, 96, 83], [613, 253, 72, 43]],
 [[527, 15

In [ ]:
file_paths

In [37]:
root_dir  = '../input/tensorflow-great-barrier-reef'
class Dataset(Dataset):    
    def __init__(self, root_dir, file_paths, transform=None):
        train_path = f'{root_dir}/train.csv'
        train = read_train(train_path)
        bboxes = 
        self.file_paths = list(file_paths)
        self.bboxes = list(train.list_annotations)
        self.transform = transform
         
    def __len__(self):
        return len(self.file_paths)
 
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        bboxes = self.bboxes[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image, bboxes = self.transform(image=image, bboxes=bboxes)
        return image, bboxes

- Kaggleのページ上でディスプレイされている画像の中でヒトデが多い画像を探す

In [ ]:
max_annots = 0
max_id = "" 
video_id = 0
for video_frame in range(1001, 1020):
    image_id = f'{video_id}-{video_frame}'
    len_annots = train[train['image_id']==image_id]['n_annotations'].item()
    if len_annots > max_annots:
        max_annots = len_annots
        max_id = image_id

print(max_id)
print(max_annots)

#  サンプルデータ読み込み
- 可視化参考元
https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507
- image_id: 0-1001, 1-10001

In [ ]:
import cv2
from PIL import Image
image1_path = '../input/tensorflow-great-barrier-reef/train_images/video_0/1001.jpg'
image2_path = '../input/tensorflow-great-barrier-reef/train_images/video_1/10001.jpg'

In [ ]:
sample_ids = ['0-1001', '1-10001']
sample_bboxes = []
for sample_id in sample_ids:
    _bboxes = train[train['image_id']==sample_id]['list_annotations'].item()
    sample_bboxes.append(_bboxes)

- Augmentationをしない場合

In [ ]:
dataset = Dataset(file_paths=[image1_path, image2_path], bboxes=sample_bboxes)

- Augmentation をする場合

In [ ]:
# # augmentationを考慮したdataset
# transformer = ObjectDetectionAugmentation(config['transforms_args'])
# dataset = Dataset(file_paths=[image1_path, image2_path], bboxes=sample_bboxes, transform=transformer.transform)

- 可視化関数の定義

In [ ]:
# 可視化用関数
def draw_bboxes(img, bboxes):
    for i in range(len(bboxes)):
        box = bboxes[i]
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = x0 + int(box[2])
        y1 = y0 + int(box[3])
        cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
    return img

def draw_bboxes_predict(img, bboxes, scores, bbclasses, confthre, classes_dict):
    img = draw_bboxes(img, bboxes)
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(bbclasses[i])
        score = scores[i]
        if score < confthre:
            continue
        cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (box[0], box[1] - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img


# def display_cv2img(cv2_img):
#     out_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
#     display(Image.fromarray(out_img))

# サンプル画像の表示

In [ ]:
out_img= draw_bboxes(*dataset.__getitem__(0))
display(Image.fromarray(out_img))

- 推論時想定可視化サンプル

In [ ]:
# # Get predictions
# # bboxes, bbclasses, scores = yolox_inference(img, model, test_size)
# bboxes, bbclasses, scores = sample_bboxes, [0], [0.99]
# # Draw predictions
# out_image = draw_bboxes_predict(sample_img, bboxes, scores, bbclasses, 0.0, {0: 'hitode'})

# # Since we load image using OpenCV we have to convert it 
# out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
# display(Image.fromarray(out_image))

# Augmentation
- 参考
    - https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/ 
    - https://github.com/albumentations-team/albumentations

In [ ]:
config['transforms_args']

In [ ]:
transformer = ObjectDetectionAugmentation(config['transforms_args'])

In [ ]:
sample_imgs, sample_bboxes = [], []
for idx in [0,1]:
    sample_img, sample_bbox = dataset.__getitem__(idx)
    sample_imgs.append(sample_img)
    sample_bboxes.append(sample_bbox)

TODO: サンプルを書き換える

In [ ]:
dataset_transform = Dataset(file_paths=[image1_path, image2_path], bboxes=sample_bboxes, transform=transformer.transform)

In [ ]:
%%time
# 一枚だけ変換
_img, _bboxes = transformer.transform(sample_img, sample_bbox)
#可視化
display(Image.fromarray(draw_bboxes(_img, _bboxes)))

In [ ]:
%%time
#　リストを全て変換
image_list, bboxes_list = transformer.transform_data(sample_imgs, sample_bboxes)
#可視化
for _i, _b in zip(image_list, bboxes_list):
    display(Image.fromarray(draw_bboxes(_i, _b)))

# 実行時間調査

In [ ]:
%%time
# 単純なループ

for _img, _bboxes in zip(sample_imgs, sample_bboxes):
    draw_bboxes(*transformer.transform(_img, _bboxes))

In [ ]:
%%time
# frompyfucによる変換

image_list, bboxes_list = np.frompyfunc(transformer.transform, 2, 2)(pd.Series(sample_imgs), pd.Series(sample_bboxes))

In [ ]:
# np.vectorize?
# np.arrayを引数に持つ関数のvectorize

- Augmentationテスト

In [ ]:
import albumentations as A

transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', min_area=100, min_visibility=0.1, label_fields=[]))

transformed = transform(image=sample_img, bboxes=sample_bbox)
transformed_img = transformed['image']
transformed_bboxes = transformed['bboxes']

transformed_img = draw_bboxes(transformed_img, transformed_bboxes)
display(Image.fromarray(transformed_img))

In [ ]:
image_id

In [ ]:
dataset = Dataset(file_paths=[image1_path, image2_path], bboxes=sample_bboxes)
img_list, bboxes_list = dataset.__getitem__(0)

img_list.shape

In [ ]:
from torch.utils.data import Dataset
class Dataset(Dataset):
     
    def __init__(self, file_paths, bboxes, transform=None):
        self.file_paths = file_paths
        self.bboxes = bboxes
        self.transform = transform
         
    def __len__(self):
        return len(self.file_paths)
 
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        bboxes = self.bboxes[idx]
 
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image, bboxes = self.transform(image=image, bboxes=bboxes)
        return image, bboxes